In [1]:
import math
import pandas as pd
import os

In [2]:
# Configurations
pd.set_option('display.max_columns', None)

In [3]:
# Initialize dictionaries of dataframes

pbp = {}
NGS_pbp = {}
merged_pbp = {}

years = [18, 19, 20, 21, 22, 23]


In [4]:
# Load CSVs per year

# NOTE: You can comment out a line if you already have the data and don't want to import it
for x in years:
    pbp[x] = pd.read_csv('data/raw/pbp/play_by_play_20' + str(x) + '.csv')
    NGS_pbp[x] =  pd.read_csv('data/raw/pbp_participation/pbp_participation_20' + str(x) + '.csv')

# print(pbp[18].head())

# print(NGS_pbp[18].head())

print("done")


/var/folders/z2/rm0rvjws51g2b4p7_0ns_31w0000gp/T/ipykernel_71676/788212044.py:5: DtypeWarning: Columns (40,41,44,45,49,50,51,52,179,180,182,183,187,188,191,192,193,194,195,196,197,198,199,200,203,204,205,206,207,208,211,212,213,214,215,216,217,218,219,220,222,224,226,227,228,229,230,231,232,240,241,242,246,247,248,249,250,251,252,253,254,255,256,258,259,260,262,263,264,265,266,267,268,269,270,272,273,274,277,278,283,284) have mixed types. Specify dtype option on import or set low_memory=False.
  pbp[x] = pd.read_csv('data/raw/pbp/play_by_play_20' + str(x) + '.csv')
/var/folders/z2/rm0rvjws51g2b4p7_0ns_31w0000gp/T/ipykernel_71676/788212044.py:5: DtypeWarning: Columns (45,179,180,182,183,189,190,193,194,197,198,203,204,205,206,209,210,218,219,220,233,234,235,236,237,238,248,249,253,254,255,260,262,263,283,284) have mixed types. Specify dtype option on import or set low_memory=False.
  pbp[x] = pd.read_csv('data/raw/pbp/play_by_play_20' + str(x) + '.csv')
/var/folders/z2/rm0rvjws51g2b4p7_

done


In [5]:
# Dimensions before cleaning

for x in years:
    print("For pbp data in 20" + str(x) + ", there are " + str(len(pbp[x])) + " rows and "
          + str(len(pbp[x].columns)) + " columns.")

for x in years:
    print("For pbp_participation data in 20" + str(x) + ", there are " + str(len(NGS_pbp[x])) + " rows and "
          + str(len(NGS_pbp[x].columns)) + " columns.")


For pbp data in 2018, there are 47109 rows and 372 columns.
For pbp data in 2019, there are 47258 rows and 372 columns.
For pbp data in 2020, there are 47705 rows and 372 columns.
For pbp data in 2021, there are 49922 rows and 372 columns.
For pbp data in 2022, there are 49434 rows and 372 columns.
For pbp data in 2023, there are 49665 rows and 372 columns.
For pbp_participation data in 2018, there are 47875 rows and 20 columns.
For pbp_participation data in 2019, there are 48034 rows and 20 columns.
For pbp_participation data in 2020, there are 48513 rows and 20 columns.
For pbp_participation data in 2021, there are 50714 rows and 20 columns.
For pbp_participation data in 2022, there are 50150 rows and 20 columns.
For pbp_participation data in 2023, there are 47160 rows and 20 columns.


# Drop Columns

Both datasets have substantially more columns we could ever need, particularly `pbp`. Most columns contain extraneous and niche information. I'm going to drop as few columns as possible since it's easier to drop than re-add, but the process of re-inserting droped columns is not very difficult.

Note: we should keep `play_id` and `game_id` since they're unique values that we can use to INNER JOIN these two dataframes together into one.

Also note: We may want to drop rows *then* columns, as we may want to drop niche plays (e.g. safeties, laterals, punt return TDs, turnovers, etc.) then drop their corresponding columns.

### Drop Columns for pbp_participation

Drop columns that contain player IDs number of players on offense/defense (it's almost always 11, who knew).

In [6]:
# Choose columns to drop

ngs_dropColumns = ["players_on_play", 
               "offense_players", 
               "defense_players",
               "n_offense",
               "n_defense"]

In [7]:
# Execute column drops for each dataframe in pbp_particpation

for x in years:
    NGS_pbp[x] = NGS_pbp[x].drop(columns = ngs_dropColumns)

In [8]:
# Now check dimensions after cleaning 

for x in years:
    print("For pbp_participation data in 20" + str(x) + ", there are " + str(len(NGS_pbp[x])) + " rows and "
          + str(len(NGS_pbp[x].columns)) + " columns.")

For pbp_participation data in 2018, there are 47875 rows and 15 columns.
For pbp_participation data in 2019, there are 48034 rows and 15 columns.
For pbp_participation data in 2020, there are 48513 rows and 15 columns.
For pbp_participation data in 2021, there are 50714 rows and 15 columns.
For pbp_participation data in 2022, there are 50150 rows and 15 columns.
For pbp_participation data in 2023, there are 47160 rows and 15 columns.


### Drop Columns for pbp

Unfortunately, since there are so 300+ columns, this is obviously a much more difficult process. I'll organize which columns to drop by "theme" as I think this will be the clearest and easiest way to do so.

In [9]:
# Choose columns to drop based on "theme"

gameInfo = ["old_game_id", "week", "game_date", "season", "season_type", "week"] # consider dropping team/posession info

metaData = ["start_time", "time_of_day", "stadium", 
            "weather", "nfl_api_id"]

timeLeft = ["half_seconds_remaining", "game_half", "quarter_end", "drive"] 

situational = ["sp", "goal_to_go", "yrdln", "ydsnet"] 

# TODO: Consider keeping "play_type" and dropping rows where it's "kickoff" instead
#
# TODO: Same with "qb_kneel" or "qb_spike", consider dropping rows where it's 1 (i.e. true) 
# instead of dropping altogether
playInfo = ["desc", "shotgun", "qb_dropback", "qb_kneel", "qb_spike", 
            "incomplete_pass", "complete_pass"]

playType = ["extra_point_attempt",	"two_point_attempt", "field_goal_attempt",	
            "kickoff_attempt",	"punt_attempt", "rush_attempt", "pass_attempt"]

kicking_and_XP = ["field_goal_result", "kick_distance", 
                  "extra_point_result", "two_point_conv_result",
                 "extra_point_prob", "two_point_conversion_prob", "home_opening_kickoff"]

timeouts = ["home_timeouts_remaining",	"away_timeouts_remaining",
            "timeout", "timeout_team",
            "defteam_timeouts_remaining"]

tdInfo = ["td_team", "td_player_name",	"td_player_id"]

scoreDiff = ["posteam_score_post", "defteam_score_post"]

ptsTypeProb = ["opp_fg_prob", "opp_safety_prob", "opp_td_prob", 
               "fg_prob", "safety_prob", "td_prob"]

epa = ["total_home_epa", "total_away_epa", "total_home_rush_epa", 
       "total_away_rush_epa", "total_home_pass_epa", "total_away_pass_epa", 
       "air_epa", "yac_epa", "comp_air_epa", "comp_yac_epa", 
       "total_home_comp_air_epa", "total_away_comp_air_epa", 
       "total_home_comp_yac_epa", "total_away_comp_yac_epa", 
       "total_home_raw_air_epa", "total_away_raw_air_epa", 
       "total_home_raw_yac_epa", "total_away_raw_yac_epa"]

wpa = ["vegas_wpa", "vegas_home_wpa", "home_wp_post", "away_wp_post", 
       "vegas_wp", "vegas_home_wp", "total_home_rush_wpa", "total_away_rush_wpa", 
       "total_home_pass_wpa", "total_away_pass_wpa", "air_wpa", "yac_wpa", 
       "comp_air_wpa", "comp_yac_wpa", "total_home_comp_air_wpa", 
       "total_away_comp_air_wpa", "total_home_comp_yac_wpa", 
       "total_away_comp_yac_wpa", "total_home_raw_air_wpa", "total_away_raw_air_wpa", 
       "total_home_raw_yac_wpa", "total_away_raw_yac_wpa"]

downInfo = ["first_down_pass", "first_down_penalty", "third_down_converted", 
            "third_down_failed", "fourth_down_converted", "fourth_down_failed", 
            "first_down_rush"]

punt_and_kickoff = ["punt_blocked", "touchback", "punt_inside_twenty", "punt_in_endzone", 
                    "punt_out_of_bounds", "punt_downed", "punt_fair_catch", 
                    "kickoff_inside_twenty", "kickoff_in_endzone", "kickoff_out_of_bounds", 
                    "kickoff_downed", "kickoff_fair_catch", "own_kickoff_recovery"
                    ,"own_kickoff_recovery_td"]

turnover = ["interception", "fumble_forced", "fumble_not_forced", "fumble_out_of_bounds",
           "fumble_lost", "fumble"]

defensive = ["solo_tackle", "safety", "qb_hit", "sack", "tackled_for_loss", "assist_tackle"]

touchdown = ["touchdown", "pass_touchdown", "rush_touchdown", "return_touchdown"]

lateral = ["lateral_reception", "lateral_rush", "lateral_return", "lateral_recovery",
          "lateral_receiver_player_id", "lateral_receiver_player_name", 
           "lateral_receiving_yards", "lateral_rusher_player_id", 
           "lateral_rusher_player_name", "lateral_rushing_yards", 
           "lateral_sack_player_id", "lateral_sack_player_name",
          "lateral_interception_player_name","lateral_punt_returner_player_id",
           "lateral_punt_returner_player_name", "lateral_interception_player_id",
           "lateral_kickoff_returner_player_id", "lateral_kickoff_returner_player_name"]

offPlayerID = ["passer_player_id", "passer_player_name", "passing_yards", 
               "receiver_player_id", "receiver_player_name", "receiving_yards", 
               "rusher_player_id", "rusher_player_name", "rushing_yards"]

puntPlayerID = ["punt_returner_player_id", "punt_returner_player_name",
               "punter_player_id", "punter_player_name"]

kickingPlayerID = ["kickoff_returner_player_name",	"kickoff_returner_player_id",
                   "kicker_player_name", "kicker_player_id",
                   "own_kickoff_recovery_player_id", "own_kickoff_recovery_player_name", 
                   "blocked_player_id", "blocked_player_name", "return_team", "return_yards"]

defensePlayerID = ["tackle_for_loss_1_player_id", "tackle_for_loss_1_player_name", 
                   "tackle_for_loss_2_player_id", "tackle_for_loss_2_player_name", 
                   "qb_hit_1_player_id", "qb_hit_1_player_name", "qb_hit_2_player_id", 
                   "qb_hit_2_player_name", "forced_fumble_player_1_team", 
                   "forced_fumble_player_1_player_id", "forced_fumble_player_1_player_name", 
                   "forced_fumble_player_2_team", "forced_fumble_player_2_player_id", 
                   "forced_fumble_player_2_player_name", "solo_tackle_1_team", "solo_tackle_2_team", 
                   "solo_tackle_1_player_id", "solo_tackle_2_player_id", "solo_tackle_1_player_name",
                   "solo_tackle_2_player_name", "assist_tackle_1_player_id", "assist_tackle_1_player_name", 
                   "assist_tackle_1_team", "assist_tackle_2_player_id", "assist_tackle_2_player_name", 
                   "assist_tackle_2_team", "assist_tackle_3_player_id", "assist_tackle_3_player_name", 
                   "assist_tackle_3_team", "assist_tackle_4_player_id", "assist_tackle_4_player_name", 
                   "assist_tackle_4_team", "tackle_with_assist", "tackle_with_assist_1_player_id", 
                   "tackle_with_assist_1_player_name", "tackle_with_assist_1_team", 
                   "tackle_with_assist_2_player_id", "tackle_with_assist_2_player_name",
                   "tackle_with_assist_2_team", "pass_defense_1_player_id", "pass_defense_1_player_name", 
                   "pass_defense_2_player_id", "pass_defense_2_player_name", "fumbled_1_team", 
                   "fumbled_1_player_id", "fumbled_1_player_name", "fumbled_2_player_id", 
                   "fumbled_2_player_name", "fumbled_2_team", "fumble_recovery_1_team", 
                   "fumble_recovery_1_yards", "fumble_recovery_1_player_id", 
                   "fumble_recovery_1_player_name", "fumble_recovery_2_team", "fumble_recovery_2_yards", 
                   "fumble_recovery_2_player_id", "fumble_recovery_2_player_name", "sack_player_id", 
                   "sack_player_name", "half_sack_1_player_id", "half_sack_1_player_name", 
                   "half_sack_2_player_id", "half_sack_2_player_name", "interception_player_id",
                   "interception_player_name"]

ref = ["replay_or_challenge", "replay_or_challenge_result",	"penalty_type"]

safety = ["defensive_two_point_attempt", "defensive_two_point_conv", 
          "defensive_extra_point_attempt", "defensive_extra_point_conv", 
          "safety_player_name", "safety_player_id"]

cpoe = ["cp", "cpoe"]

series = ["series", "series_success", "series_result", "order_sequence"]

playClock = ["play_clock", "play_deleted"]

stPlay = ["special_teams_play",	"st_play_type"]

drive = ["end_clock_time", "end_yard_line", "fixed_drive", "fixed_drive_result", 
         "drive_real_start_time", "drive_play_count", "drive_time_of_possession", 
         "drive_first_downs", "drive_inside20", "drive_ended_with_score", 
         "drive_quarter_start", "drive_quarter_end", "drive_yards_penalized", 
         "drive_start_transition", "drive_end_transition", "drive_game_clock_start", 
         "drive_game_clock_end", "drive_start_yard_line", "drive_end_yard_line", 
         "drive_play_id_started", "drive_play_id_ended"]

gameScoreData = ["away_score", "home_score", "location", "result", "total", 
                 "spread_line", "total_line", "div_game"]

weather_stadium_conditions = ["roof", "surface", "temp", "wind", 
                              "stadium_id", "game_stadium"]

coaches = ["home_coach", "away_coach"]


abortedPlay = ["aborted_play", "success"]

offJerseys = ["passer", "passer_jersey_number", "rusher", 
                 "rusher_jersey_number", "receiver", "receiver_jersey_number",
             "passer_id", "rusher_id", "receiver_id", "name", "jersey_number", "id"]

playEncoding = ["pass", "rush", "first_down", "special", "play"]

fantasy = ["fantasy_player_name", "fantasy_player_id", "fantasy", "fantasy_id"]

exp_epa = ["qb_epa", "xyac_epa", "xyac_mean_yardage", "xyac_median_yardage", 
           "xyac_success", "xyac_fd", "xpass", "pass_oe"]

# Should this be dropped? How would this affect EPA?
penalty = ["penalty", "penalty_player_id",	"penalty_player_name",	"penalty_yards", "penalty_team"]

In [10]:

# Dictionary mapping list names to their respective lists
columnTypes = {
    "playInfo": playInfo,
    "metaData": metaData,
    "playType": playType,
    "timeLeft" : timeLeft,
    "situational": situational,
    "kicking_and_XP": kicking_and_XP,
    "timeouts": timeouts,
    "tdInfo": tdInfo,
    "scoreDiff": scoreDiff,
    "ptsTypeProb": ptsTypeProb,
    "epa": epa,
    "wpa": wpa,
    "downInfo": downInfo,
    "punt_and_kickoff": punt_and_kickoff,
    "turnover": turnover,
    "defensive": defensive,
    "touchdown": touchdown,
    "lateral": lateral,
    "offPlayerID": offPlayerID,
    "puntPlayerID": puntPlayerID,
    "kickingPlayerID": kickingPlayerID,
    "defensePlayerID": defensePlayerID,
    "ref": ref,
    "safety": safety,
    "cpoe": cpoe,
    "series": series,
    "playClock": playClock,
    "stPlay": stPlay,
    "drive": drive,
    "gameScoreData": gameScoreData,
    "weather_stadium_conditions": weather_stadium_conditions,
    "coaches": coaches,
    "abortedPlay": abortedPlay,
    "offJerseys": offJerseys,
    "playEncoding": playEncoding,
    "fantasy": fantasy,
    "exp_epa": exp_epa,
    "penalty": penalty,
}


In [11]:
for x in years:
    for c in columnTypes.values():
        pbp[x] = pbp[x].drop(columns = c)

In [12]:
# Test on one dataframe

'''
pbp[18] = pd.read_csv('data/raw/pbp/play_by_play_2018.csv')

for c in columnTypes.values():
        pbp[18] = pbp[18].drop(columns = c)

pbp[18].to_csv('data/test_2018.csv')
'''

"\npbp[18] = pd.read_csv('data/raw/pbp/play_by_play_2018.csv')\n\nfor c in columnTypes.values():\n        pbp[18] = pbp[18].drop(columns = c)\n\npbp[18].to_csv('data/test_2018.csv')\n"

In [13]:
# Dimensions after cleaning
# There used to be like 360+ columns and 47k+ rows in any given dataframe

for x in years:
    print("For pbp data in 20" + str(x) + ", there are " + str(len(pbp[x])) + " rows and "
          + str(len(pbp[x].columns)) + " columns.")

For pbp data in 2018, there are 47109 rows and 47 columns.
For pbp data in 2019, there are 47258 rows and 47 columns.
For pbp data in 2020, there are 47705 rows and 47 columns.
For pbp data in 2021, there are 49922 rows and 47 columns.
For pbp data in 2022, there are 49434 rows and 47 columns.
For pbp data in 2023, there are 49665 rows and 47 columns.


# Clean Rows

Eliminate plays (i.e. rows) that are unwanted. These plays include clock management plays (e.g. spikes and kneels), outliers (e.g. safeties), special team plays, XPs, null values, etc.

NOTE: This doesn't account for safeties. I may have accidentally dropped a column that specified if a play was a safety. However, considering how few safeties there are and how safeties are accounted for in EPA already, I'm not sure if accounting for this is a priority.

### Drop NGS_pbp Rows

Eliminate rows that have NaN on in any column. The rationale is that we should assume each entry is either complete, and if it is incomplete then the row either represents a non-conventional play (e.g. special teams, kneel, etc.) or is useless anyway.

In [14]:
# Check levels for formations, see if null values return NULL, None, NaN, etc.
# dropna() should drop any row regardless if it is NULL, None, or NaN
# so theorhetically we don't need this step. I just want to be safe though.

print(NGS_pbp[18]['offense_formation'].unique())


[nan 'I_FORM' 'SINGLEBACK' 'SHOTGUN' 'JUMBO' 'EMPTY' 'PISTOL' 'WILDCAT']


In [15]:
# Drop Rows

for x in years:
    df = NGS_pbp[x]
    NGS_pbp[x] = df.dropna()

In [16]:
for x in years:
    print("For pbp_participation data in 20" + str(x) + ", there are " + str(len(NGS_pbp[x])) + " rows and "
          + str(len(NGS_pbp[x].columns)) + " columns.")

# This drops more rows than I like, I have no idea why this drops so many more rows
# than the similar process in pbp.
# There shouldn't be nearly 30k punts and/or incomplete entries, right?

For pbp_participation data in 2018, there are 17783 rows and 15 columns.
For pbp_participation data in 2019, there are 17730 rows and 15 columns.
For pbp_participation data in 2020, there are 18142 rows and 15 columns.
For pbp_participation data in 2021, there are 18849 rows and 15 columns.
For pbp_participation data in 2022, there are 18155 rows and 15 columns.
For pbp_participation data in 2023, there are 16000 rows and 15 columns.


## Drop pbp Rows

In [17]:
# First see the levels of play types

print(pbp[18]['play_type'].unique())

# I don't want to use this column because already covers
# a lot of the plays in `play_type_nfl`.
# print(pbp[18]['play_type_nfl'].unique())



[nan 'kickoff' 'no_play' 'pass' 'run' 'punt' 'field_goal' 'qb_kneel'
 'extra_point' 'qb_spike']


In [18]:
# Create list of plays we want to drop

play_type_drop = [float('nan'), 'kickoff', 'no_play', 'punt', 'field_goal', 'qb_kneel', 'qb_spike', 'extra_point']

In [19]:
# Drop Rows

for x in years:
    df = pbp[x]
    pbp[x] = df[~df['play_type'].isin(play_type_drop)]

In [20]:
# Dimensions after cleaning
# There used to be like 360+ columns and 47k+ rows in any given dataframe

for x in years:
    print("For pbp data in 20" + str(x) + ", there are " + str(len(pbp[x])) + " rows and "
          + str(len(pbp[x].columns)) + " columns.")

For pbp data in 2018, there are 33292 rows and 47 columns.
For pbp data in 2019, there are 33508 rows and 47 columns.
For pbp data in 2020, there are 34238 rows and 47 columns.
For pbp data in 2021, there are 35766 rows and 47 columns.
For pbp data in 2022, there are 35430 rows and 47 columns.
For pbp data in 2023, there are 35600 rows and 47 columns.


# Export Data

In [21]:
# Export pbp to CSVs
for x in years:
    pbp[x].to_csv('cleanData/pbp/pbp_clean_20' + str(x) + '.csv')

print("Done")

Done


In [22]:
# Export pbp_participation to CSVs
for x in years:
    NGS_pbp[x].to_csv('cleanData/pbp_participation/pbp_part_clean_20' + str(x) + '.csv')

print("Done")

Done


# Merge Cleaned Data

Join the dataframes based on `play_id` AND `old_game_id`.

Note: it appears `game_id` and `nflverse_game_id` are the same, so we could merge on that and drop one of them.

In [23]:
# Initialize dictionaries of dataframes

pbp_cleaned = {}
NGS_pbp_cleaned = {}
merged_pbp = {}

In [24]:
# Reload clean CSVs per year
# Can skip this step if you'd like to avoid creating new objects

# NOTE: You can comment out a line if you already have the data and don't want to import it
for x in years:
    pbp_cleaned[x] = pd.read_csv('cleanData/pbp/pbp_clean_20' + str(x) + '.csv')
    NGS_pbp_cleaned[x] =  pd.read_csv('cleanData/pbp_participation/pbp_part_clean_20' + str(x) + '.csv')

print("Done")

Done


In [25]:
# See columns
# TODO: ensure we can avoid duplicating common columns

print(pbp_cleaned[18].columns)
print(NGS_pbp_cleaned[18].columns)


Index(['Unnamed: 0', 'play_id', 'game_id', 'old_game_id', 'home_team',
       'away_team', 'season_type', 'week', 'posteam', 'posteam_type',
       'defteam', 'side_of_field', 'yardline_100', 'game_date',
       'quarter_seconds_remaining', 'game_seconds_remaining', 'qtr', 'down',
       'time', 'ydstogo', 'play_type', 'yards_gained', 'no_huddle',
       'qb_scramble', 'pass_length', 'pass_location', 'air_yards',
       'yards_after_catch', 'run_location', 'run_gap',
       'posteam_timeouts_remaining', 'total_home_score', 'total_away_score',
       'posteam_score', 'defteam_score', 'score_differential',
       'score_differential_post', 'no_score_prob', 'ep', 'epa', 'wp', 'def_wp',
       'home_wp', 'away_wp', 'wpa', 'season', 'play_type_nfl',
       'out_of_bounds'],
      dtype='object')
Index(['Unnamed: 0', 'nflverse_game_id', 'old_game_id', 'play_id',
       'possession_team', 'offense_formation', 'offense_personnel',
       'defenders_in_box', 'defense_personnel', 'number_of_pass

In [26]:
# Run merge
# We want to merge right because we only want plays where NGS data exists

for x in years:
    merged_pbp[x] = pd.merge(pbp_cleaned[x], NGS_pbp_cleaned[x], on=['play_id', 'old_game_id'], how = 'right', suffixes=('_pbp', '_pbp_part'))

In [27]:
# Dimensions after merging
# There should be quite a few more columns added especially since there are redundancies
# Number of rows should also be no greater than the number of rows in NGS_pbp_cleaned[x]

for x in years:
    print("For the merged dataset in 20" + str(x) + ", there are " + str(len(merged_pbp[x])) + " rows and "
          + str(len(merged_pbp[x].columns)) + " columns.")

For the merged dataset in 2018, there are 17783 rows and 62 columns.
For the merged dataset in 2019, there are 17730 rows and 62 columns.
For the merged dataset in 2020, there are 18142 rows and 62 columns.
For the merged dataset in 2021, there are 18849 rows and 62 columns.
For the merged dataset in 2022, there are 18155 rows and 62 columns.
For the merged dataset in 2023, there are 16000 rows and 62 columns.


In [28]:
# Export merged data to CSVs
for x in years:
    merged_pbp[x].to_csv('cleanData/merged/pbp_merged_20' + str(x) + '.csv')

print("Done")

Done


In [30]:
# Testing cell
#print(NGS_pbp_cleaned[18].head())
print(merged_pbp[18].columns)



Index(['Unnamed: 0_pbp', 'play_id', 'game_id', 'old_game_id', 'home_team',
       'away_team', 'season_type', 'week', 'posteam', 'posteam_type',
       'defteam', 'side_of_field', 'yardline_100', 'game_date',
       'quarter_seconds_remaining', 'game_seconds_remaining', 'qtr', 'down',
       'time', 'ydstogo', 'play_type', 'yards_gained', 'no_huddle',
       'qb_scramble', 'pass_length', 'pass_location', 'air_yards',
       'yards_after_catch', 'run_location', 'run_gap',
       'posteam_timeouts_remaining', 'total_home_score', 'total_away_score',
       'posteam_score', 'defteam_score', 'score_differential',
       'score_differential_post', 'no_score_prob', 'ep', 'epa', 'wp', 'def_wp',
       'home_wp', 'away_wp', 'wpa', 'season', 'play_type_nfl', 'out_of_bounds',
       'Unnamed: 0_pbp_part', 'nflverse_game_id', 'possession_team',
       'offense_formation', 'offense_personnel', 'defenders_in_box',
       'defense_personnel', 'number_of_pass_rushers', 'ngs_air_yards',
       'time_to